In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import imageio
import scipy
from sklearn import preprocessing
from scipy import signal
import timit_utils as tu
import timit_utils.audio_utils as au
import timit_utils.drawing_utils as du
import librosa
import pywt
import random
from python_speech_features import mfcc
from python_speech_features import logfbank
from scipy.signal import butter, lfilter
import speech_recognition as sr
import re

corpus = tu.Corpus('TIMIT')
train = corpus.train
test = corpus.test
corpus.train, corpus.test

(<timit_utils.core.SubCorpus at 0x1acdf3483c8>,
 <timit_utils.core.SubCorpus at 0x1ace90821d0>)

In [3]:
data_set = []
label_id = []
for p in range(len(train.people)):
    count = 0
    for s in range(len(train.person_by_index(p).sentences)):
        sentence = train.person_by_index(p).sentence_by_index(s)
        label_id.append(train.person_by_index(p).name)
        data_set.append(sentence)
for p in range(len(test.people)):
    count = 0
    for s in range(len(test.person_by_index(p).sentences)):
        sentence = test.person_by_index(p).sentence_by_index(s)
        label_id.append(test.person_by_index(p).name)
        data_set.append(sentence)

le1 = preprocessing.LabelEncoder()
le1.fit(label_id)
label_id = le1.transform(label_id)
N = 20 # number of speakers being considered
sub_data_set = []
speak_ind = []
S = [120,439,531,191,481,179,309,201,33,90,602,200,6,451,444,337,466,599,214,611]
for ind1 in range(N):
    ind2 = S[ind1]
    speak_ind.append(ind2)
act_ind = []
for ID in speak_ind:
    act_ind= np.where(label_id == ID)
    sub_data_set = sub_data_set+data_set[act_ind[0][0]:act_ind[0][-1]+1]

In [ ]:
cf_array = [10,100,200,500,1000,2000,3000,4000,5000,6000,7000,8000]
w_performances = []
s_performances = []
for cf in cf_array:
    print('Cutoff Frequency is {}.'.format(cf))
    wrongs = []
    corrects = []
    for s0 in sub_data_set:
        sentence_w = s0.words_df.index.values
        sentence_a = s0.raw_audio
        if cf == 8000:
            d = sentence_a
        else:
            b, a = butter(10, cf/(16000/2), btype='low', analog=False)
            d = lfilter(b,a,sentence_a)
        sentence_a = np.int16(d*0.3*32767)
        scipy.io.wavfile.write('temp.wav', 16000, sentence_a)
        r = sr.Recognizer()
        with sr.AudioFile('temp.wav') as source:
            audio = r.record(source)  # read the entire audio file
        wordList = []
        try:
            words = r.recognize_google(audio)
            wordList = re.sub("[^\w]", " ",  words).lower().split()
        except:
            pass
        wrong = 0
        correct = 0
        for word in sentence_w:
            if word in wordList:
                correct = correct+1
            else:
                wrong = wrong+1
        wrongs = wrongs+[wrong]
        corrects = corrects+[correct]
    w_correct = sum(corrects)/(sum(corrects)+sum(wrongs))*100
    s_correct = wrongs.count(0)/(N*10)*100
    w_performances = w_performances + [w_correct]
    s_performances = s_performances + [s_correct]
print(w_performances)
print(s_performances)

Cutoff Frequency is 10.


C:\Users\jessi\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: overflow encountered in multiply
  app.launch_new_instance()


Cutoff Frequency is 100.
Cutoff Frequency is 200.
Cutoff Frequency is 500.
